# 3rd Period - Neural Network from scratch

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("dataset/star_classification.csv")

In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X = df.drop(columns=['class']).values.astype(float)
lb = LabelBinarizer()
Y = lb.fit_transform(df['class'])

    
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [4]:
class TestNeuralNetwork:
    def __init__(self, layers_shape: list[int]) -> None:
        self.layers_shape = layers_shape
        self.n_layers = len(layers_shape)
        self.weights = [self.weights_initialization(layers_shape[i], layers_shape[i+1]) for i in range(self.n_layers-1)]

    def sigmoid(self, x: np.ndarray) -> np.ndarray:
        return 1/(1 + np.exp(-x))

    def sigmoid_derivative(self, x: np.ndarray) -> np.ndarray:
        s = self.sigmoid(x)
        return s * (1 - s)

    def weights_initialization(self, n_in: int, n_out: int) -> np.ndarray:
        return np.random.randn(n_in, n_out) * np.sqrt(2. / n_in)

    def feed_forward(self, x: np.ndarray):
        a = x
        activations = [a]
        zs = []
        for w in self.weights:
            z = a.dot(w)
            zs.append(z)
            a = self.sigmoid(z)
            activations.append(a)
        return activations, zs

    def loss(self, out: np.ndarray, y: np.ndarray) -> float:
        eps = 1e-8
        return -np.sum(y * np.log(out + eps)) / y.shape[0]

    def back_propagation(self, x: np.ndarray, y: np.ndarray, alpha: float = 0.01) -> None:
        activations, zs = self.feed_forward(x)
        deltas = [None] * (self.n_layers - 1)
        deltas[-1] = (activations[-1] - y) * self.sigmoid_derivative(zs[-1])
        for l in range(self.n_layers-3, -1, -1):
            deltas[l] = (deltas[l+1].dot(self.weights[l+1].T)) * self.sigmoid_derivative(zs[l])
        for l in range(len(self.weights)):
            a_prev = activations[l].reshape(-1,1)
            d = deltas[l].reshape(1,-1)
            self.weights[l] -= alpha * a_prev.dot(d)

    def train(self, X: np.ndarray, Y: np.ndarray, alpha: float, epoch: int):
        acc = []
        losss = []
        for j in range(epoch):
            l = []
            correct = 0
            for i in range(len(X)):
                out, _ = self.feed_forward(X[i])
                l.append(self.loss(out[-1], Y[i]))
                self.back_propagation(X[i], Y[i], alpha)
                if np.argmax(out[-1]) == np.argmax(Y[i]):
                    correct += 1
            print(f"epochs: {j+1} ======== acc: {correct/len(X)*100:.2f}")
            acc.append(correct/len(X)*100)
            losss.append(sum(l)/len(X))
        return acc, losss, self.weights

    def predict(self, x: np.ndarray) -> int:
        out, _ = self.feed_forward(x)
        return np.argmax(out[-1])


In [5]:

nn = TestNeuralNetwork([X_train.shape[1], 32, 16, Y_train.shape[1]])
acc, loss, weights = nn.train(X_train, Y_train, alpha=0.05, epoch=15)

preds = [nn.predict(x) for x in X_test]
y_true = np.argmax(Y_test, axis=1)
accuracy = np.mean(np.array(preds) == y_true)
print(f"Accuracy: {accuracy:.2%}")
              
         

epochs: 1 ======== acc: 89.60
epochs: 2 ======== acc: 93.80
epochs: 3 ======== acc: 94.44
epochs: 4 ======== acc: 94.88


C:\Users\Notebook\AppData\Local\Temp\ipykernel_27104\2525042316.py:8: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-x))


epochs: 5 ======== acc: 95.16
epochs: 6 ======== acc: 95.32
epochs: 7 ======== acc: 95.42
epochs: 8 ======== acc: 95.53
epochs: 9 ======== acc: 95.61
epochs: 10 ======== acc: 95.70
epochs: 11 ======== acc: 95.79
epochs: 12 ======== acc: 95.87
epochs: 13 ======== acc: 95.93
epochs: 14 ======== acc: 95.98
epochs: 15 ======== acc: 96.05
Accuracy: 96.14%
